In [4]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [49]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

# nltk.download("punkt")
import sys

sys.path.append("../")


from utils.dataset import CBOW_Dataset

from scipy.spatial.distance import cosine

In [6]:
data = CBOW_Dataset("WikiText2", "train", "../data/")
data_iter = data.data_iter

In [7]:
def get_sent_tokens_from_iterator(data_iter):
    all_sent_tokens = []
    for line in data_iter:
        sent_tokens = []
        line_lower = line.lower()
        sentences = sent_tokenize(line_lower)
        for sent in sentences:
            sent_tokens.append(sent.split())

        if len(sent_tokens) > 0:
            all_sent_tokens.extend(sent_tokens)
    return all_sent_tokens


sent_tokens = get_sent_tokens_from_iterator(data_iter)
len(sent_tokens)

86861

In [14]:
n_tokens = 0
for x in sent_tokens:
    n_tokens += len(x)

n_tokens / 1000000

2.055104

## Training Word2Vec with Gensim

In [3]:
from gensim.models import Word2Vec

# https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models

In [21]:
model = Word2Vec(
    sentences=sent_tokens,
    vector_size=300,
    window=4,
    min_count=50,
    workers=4,
    sg=1,
    hs=1,
    epochs=5,
    compute_loss=True,
)
model.get_latest_training_loss()
# model.save("word2vec.model")

44667196.0

In [22]:
model.get_latest_training_loss()

44667196.0

In [82]:
model.wv.most_similar("1", topn=10)

[('4', 0.7374173998832703),
 ('3', 0.7292033433914185),
 ('2', 0.7034738063812256),
 ('6', 0.6723881959915161),
 ('5', 0.638543963432312),
 ('8', 0.6239730715751648),
 ('7', 0.6155374050140381),
 ('9', 0.6041812896728516),
 ('0', 0.5962584018707275),
 ('10', 0.5761900544166565)]

In [60]:
king_v = model.wv["king"]
queen_v = model.wv["queen"]
woman_v = model.wv["woman"]
man_v = model.wv["man"]

king_v = king_v / ((king_v ** 2).sum() ** (1 / 2))
queen_v = queen_v / ((queen_v ** 2).sum() ** (1 / 2))
woman_v = woman_v / ((woman_v ** 2).sum() ** (1 / 2))
man_v = man_v / ((man_v ** 2).sum() ** (1 / 2))

In [59]:
(king_v ** 2).sum()

0.9999999

In [63]:
new_v = (king_v - man_v) + woman_v

In [62]:
cosine(new_v, queen_v)

0.5770814418792725